<h1>Loading the pictures</h1>
  We are considering the following:
  <table>
    <thead>
      <tr>
        <th>Category</th>
        <th>Types</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td rowspan="3">Happy</td>
        <td>happy</td>
      </tr>
      <tr><td>wink</td></tr>
      <tr><td>normal</td></tr>
      <tr>
        <td rowspan="4">Sad</td>
        <td>sad</td>
      </tr>
      <tr><td>sleepy</td></tr>
      <tr><td>glasses</td></tr>
    </tbody>
  </table>


In [27]:
from skimage.io import imread
from os import listdir
import cv2
from keras_facenet import FaceNet


def loadImages(happy: list, sad: list) -> tuple:
  directory = "drive/MyDrive/AI/yalefaces"
  files = listdir(directory)
  inputs, outputs = [], []
  embedder = FaceNet()
  for url in files:
    extension = url.split(".")[1]
    isHappy = extension in happy
    isSad = extension in sad
    if isHappy or isSad:
      image = imread(f"{directory}/{url}")
      imageColor = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
      embeddings = embedder.extract(imageColor, threshold=0.95)
      inputs.append(embeddings[0]["embedding"])
      outputs.append(0 if isHappy else 1)
  return inputs, outputs

<h1>Evaluating the classifier</h1>

In [32]:
def multiClassEvaluation(realLabels: list, computedLabels: list, labelNames: list):
    accMean, precisionMean, recallMean, nrLabels = 0, 0, 0, len(labelNames)
    for label in labelNames:
        TP = sum([1 if real == label and computed == label else 0 for real, computed in zip(realLabels, computedLabels)])
        TN = sum([1 if real != label and computed != label else 0 for real, computed in zip(realLabels, computedLabels)])
        FP = sum([1 if real != label and computed == label else 0 for real, computed in zip(realLabels, computedLabels)])
        FN = sum([1 if real == label and computed != label else 0 for real, computed in zip(realLabels, computedLabels)])
        try:
            acc = (TP + TN) / (TP + TN + FP + FN)
        except ZeroDivisionError:
            acc = 0
        try:
            precision = TP / (TP + FP)
        except ZeroDivisionError:
            precision = 0
        try:
            recall = TP / (TP + FN)
        except ZeroDivisionError:
            recall = 0
        accMean += acc
        precisionMean += precision
        recallMean += recall
    return accMean / nrLabels, precisionMean / nrLabels, recallMean / nrLabels

<h1>Main</h1>

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier


happyCategories = ["happy", "wink", "normal"]
sadCategories = ["sad", "sleepy", "glasses"]

inputData, outputData = loadImages(happyCategories, sadCategories)
print("Loaded data")

trainInput, validInput, trainOutput, validOutput = train_test_split(inputData, outputData, train_size=8/10)

classifier = SGDClassifier(random_state=42, max_iter=2000, tol=1e-3)
classifier.fit(inputData, outputData)

validComputedOutput = list(classifier.predict(validInput))

print(f"REAL     {validOutput}")
print(f"COMPUTED {validComputedOutput}")

accuracy, precision, recall = multiClassEvaluation(validOutput, validComputedOutput, [0, 1])
print(f"accuracy {accuracy}\nprecision {precision}\nrecall {recall}")

Loaded data
REAL     [0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
COMPUTED [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
accuracy 0.631578947368421
precision 0.7666666666666666
recall 0.6818181818181819
